In [57]:
import os
import neptune
import zipfile
import shutil

# === CONFIGURATION ===
API_TOKEN = os.getenv("NEPTUNE_API_TOKEN", "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZjNlODVlNC04NzYwLTQ5ZWItYTUyYy1kY2EwNjBmYjFjODcifQ==")  # Replace with your API token or set the environment variable.
PROJECT_NAME = "sbuser/SX3M"              # Adjust to your project name.
RUN_IDS = {
    "SXM-449": "SXM-449",  # Replace with the actual run identifier if different.
    "SXM-762": "SXM-762"   # Replace with the actual run identifier if different.
}
DOWNLOAD_BASE_DIR = "./neptune_runs"

# Create base directory for downloads
os.makedirs(DOWNLOAD_BASE_DIR, exist_ok=True)

In [57]:
p =neptune.Project(project="sbuser/SX3M",
                  api_token=os.getenv("NEPTUNE_API_TOKEN", "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZjNlODVlNC04NzYwLTQ5ZWItYTUyYy1kY2EwNjBmYjFjODcifQ==")  )

models_table_d=p.fetch_models_table().to_pandas()

In [57]:
models_table=p.fetch_models_table()

lgbm=models_table.to_rows()[0]

In [57]:
model = neptune.Model(project="sbuser/SX3M", mode="read-only", with_id="SXM-XGRF",
                  api_token=os.getenv("NEPTUNE_API_TOKEN", "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZjNlODVlNC04NzYwLTQ5ZWItYTUyYy1kY2EwNjBmYjFjODcifQ==")  )

mv=model.fetch_model_versions_table().to_pandas().T

In [ ]:
run = neptune.Run(project="sbuser/SX3M", mode="read-only", with_id="SXM-449",
                  api_token=os.getenv("NEPTUNE_API_TOKEN", "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZjNlODVlNC04NzYwLTQ5ZWItYTUyYy1kY2EwNjBmYjFjODcifQ==")  )

models =run.get_attribute('model').fetch().keys()
for model_name in models:
    model_run_environment =  run['model'][model_name].fetch().keys() # dict_keys(['estimator', 'hyperoptimized_best_params', 'sys', 'tuning'])
    
    #continue for loop through model names
    run['model']['XGBoost'].fetch().keys()  #dict_keys(['estimator', 'hyperoptimized_best_params', 'sys', 'tuning'])

    model_params=run['model']['XGBoost']['estimator']['params'].fetch()  # returns one more dict that has all the types and values
    
    #next keep in model name dict
    best_parms=run['model']['XGBoost']['hyperoptimized_best_params'].fetch()
​

In [ ]:
mrun = run.fetch()['model']

run_structure = run.print_structure()

ns=run["model"]

ns.container.print_structure()
for i in ns["model"]:
    i.fetch()

def download_run_data(run_id):
    print(f"Processing run {run_id}...")
    # Initialize the run object (read-only mode)
    run = neptune.init_run(
        project=PROJECT_NAME,
        api_token=API_TOKEN,
        run=run_id,
        mode="read-only"  # Ensure we are not modifying anything.
    )
    
    # Create a subdirectory for this run
    run_dir = os.path.join(DOWNLOAD_BASE_DIR, run_id)
    os.makedirs(run_dir, exist_ok=True)

    # Download all files (artifacts & metadata)
    # Note: You might need to adjust the download_files call according to your Neptune version
    run.download_files(destination=run_dir)
    
    # Terminate the run to free resources
    run.stop()
    
    # Zip the downloaded directory
    zip_filename = os.path.join(DOWNLOAD_BASE_DIR, f"{run_id}_run_data.zip")
    print(f"Creating ZIP file: {zip_filename}")
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(run_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, run_dir))
    
    # Optionally, remove the unzipped files to save space
    shutil.rmtree(run_dir)
    print(f"Run {run_id} data has been downloaded and zipped.\n")

for run_label, run_id in RUN_IDS.items():
    download_run_data(run_id)